In [5]:
import pandas as pd
import numpy as np

In [6]:
df_train = pd.read_csv('./data/train.csv')

In [7]:
print("Reading labels ...")
print(df_train.head())

Reading labels ...
  image_name                                       tags
0    train_0                               haze primary
1    train_1            agriculture clear primary water
2    train_2                              clear primary
3    train_3                              clear primary
4    train_4  agriculture clear habitation primary road


In [9]:
X = np.array([np.load('./baseline/tmp/TMPDIR%s.jpg.npy' % str(name)) for name in df_train['image_name'].tolist()])

In [10]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}


In [11]:
label_map

{'agriculture': 16,
 'artisinal_mine': 2,
 'bare_ground': 10,
 'blooming': 8,
 'blow_down': 3,
 'clear': 6,
 'cloudy': 1,
 'conventional_mine': 4,
 'cultivation': 5,
 'habitation': 11,
 'haze': 15,
 'partly_cloudy': 13,
 'primary': 0,
 'road': 9,
 'selective_logging': 12,
 'slash_burn': 7,
 'water': 14}

In [40]:
from sklearn.metrics import fbeta_score
import xgboost as xgb

In [13]:
param = {}
param['objective'] = 'binary:softprob'
param['eta'] = 0.2
param['max_depth'] = 4
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "logloss"
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.5
param['seed'] = 1337

In [31]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [16]:
le = LabelBinarizer()

In [18]:
y = le.fit_transform(df_train['tags'])

In [24]:
plst = list(param.items())

In [27]:
# Create a validation set
x_trn, x_val, y_trn, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
xgb_c = xgb.XGBClassifier(n_estimators=100)

In [43]:
multi_target_forest = MultiOutputClassifier(xgb_c, n_jobs=-1)

In [44]:
X = X.reshape(40479,2048)

In [ ]:
multi_target_forest.fit(X, y)

In [ ]:
from sklearn.externals import joblib
joblib.dump(multi_target_forest, 'multi_target_forest.pkl') 